# US household debt indicators
> This notebook downloads, processes and charts data related to household debt from the US Federal Reserve. 

---

#### Import Python tools and Jupyter config

In [ ]:
import os
import cpi
import requests
import pandas as pd
import jupyter_black
import altair as alt
import geopandas as gpd
import altair_cnn as altcnn
from IPython.display import Image
from datawrapper import Datawrapper

In [ ]:
jupyter_black.load()
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100
pd.options.display.max_colwidth = None
alt.themes.register("cnn", altcnn.theme)
alt.themes.enable("cnn")

In [ ]:
dw_token = os.environ.get("dw_api")
dw = Datawrapper(access_token=dw_token)
today = pd.Timestamp("today").strftime("%Y-%m-%d")

---

## Fetch

#### Debt report from the [New York Fed Consumer Credit Panel/Equifax](https://www.newyorkfed.org/microeconomics/hhdc)

In [ ]:
url = "https://www.newyorkfed.org/medialibrary/interactives/householdcredit/data/xls/HHD_C_Report_2023Q4"

In [ ]:
xlsx_file = pd.read_excel(url, sheet_name="TABLE OF CONTENTS", skiprows=5, skipfooter=8)
sheet_names = pd.ExcelFile(url).sheet_names

---

#### Quarterly household debt service ratio
> This [measure](https://fred.stlouisfed.org/series/TDSP) is the ratio of total required household debt payments to total disposable income.

In [ ]:
debt_ratio_url = f"https://fred.stlouisfed.org/graph/fredgraph.csv?id=TDSP"

In [ ]:
debt_ratio_df = pd.read_csv(debt_ratio_url, names=["date", "value"], header=0).round(2)

#### Last decade

In [ ]:
recent = debt_ratio_df.query('date>"2014-12-31"').copy()

#### Export to Datawrapper [chart](https://app.datawrapper.de/chart/tXaoi/publish)

In [ ]:
debt_ratio_id = "tXaoi"
dw.add_data(
    chart_id=f"{debt_ratio_id}",
    data=recent,
)

---

#### Percentage of debt 90+ days delinquent
> Aggregate delinquency rates increased in the fourth quarter of 2023. As of December, 3.1% of outstanding debt was in some stage of delinquency, up by 0.1 percentage point from the third quarter. Still, overall delinquency rates remain 1.6 percentage points lower than the fourth quarter of 2019.

In [ ]:
del_df = (
    pd.read_excel(url, sheet_name="Page 12 Data", skiprows=3)
    .drop(["Unnamed: 8", "Unnamed: 9"], axis=1)
    .round(2)
).rename(
    columns={
        "Unnamed: 0": "quarter",
        "MORTGAGE": "mortgage",
        "HELOC": "heloc",
        "AUTO": "auto",
        "CC": "credit_card",
        "STUDENT LOAN": "student_loan",
        "OTHER": "other",
        "ALL": "all",
    }
)

#### Clean dates

In [ ]:
del_df["year"] = "20" + del_df["quarter"].str.split(":", expand=True)[0]
del_df["quarter"] = del_df["quarter"].str.split(":", expand=True)[1]

In [ ]:
quarter_to_date = {
    "Q1": "-01-01",  # January 1st
    "Q2": "-04-01",  # April 1st
    "Q3": "-07-01",  # July 1st
    "Q4": "-10-01",  # October 1st
}

In [ ]:
del_df["date"] = pd.to_datetime(
    del_df["year"].astype(str) + del_df["quarter"].map(quarter_to_date)
).dt.strftime("%Y-%m-%d")

In [ ]:
del_df.head()

#### Export to DW

In [ ]:
# https://app.datawrapper.de/chart/4xRqa/publish
del_df_id = "4xRqa"
dw.add_data(chart_id=f"{del_df_id}", data=del_df.query('year > "2014"'))

---

#### Total debt balance

#### By composition

In [ ]:
debt_df = pd.read_excel(url, sheet_name="Page 3 Data", skiprows=3).rename(
    columns={"Unnamed: 0": "quarter"}
)
debt_df.columns = debt_df.columns.str.lower().str.replace(" ", "_")

In [ ]:
debt_df["year"] = "20" + debt_df["quarter"].str.split(":", expand=True)[0]
debt_df["quarter"] = debt_df["quarter"].str.split(":", expand=True)[1]

In [ ]:
debt_df["date"] = pd.to_datetime(
    debt_df["year"].astype(str) + debt_df["quarter"].map(quarter_to_date)
).dt.strftime("%Y-%m-%d")
debt_df["date"] = pd.to_datetime(debt_df["date"])

In [ ]:
debt_df.head()

#### Function to adjust for inflation

In [ ]:
def adjust_for_inflation(row, column_name):
    # Adjusts to 2023 dollars
    year_of_data = row["date"].year
    return cpi.inflate(row[column_name], year_or_month=year_of_data, to=2023)

#### Apply inflation adjustment
> Use the non-housing measures. Other balances include retail cards and other consumer loans.

In [ ]:
columns_to_adjust = [
    # "mortgage",
    # "he_revolving",
    "auto_loan",
    "credit_card",
    "student_loan",
    "other",
    # "total",
]

In [ ]:
for column in columns_to_adjust:
    debt_df[column + "_infl_adj"] = debt_df.apply(
        adjust_for_inflation, column_name=column, axis=1
    ).round(3)

In [ ]:
debt_df.tail()

In [ ]:
# https://app.datawrapper.de/chart/U2lZt/publish
debt_df_id = "U2lZt"
dw.add_data(chart_id=f"{debt_df_id}", data=debt_df.query('year > "2014"'))